<a href="https://colab.research.google.com/github/AnoVando/MSIS/blob/master/MSIS521_IA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') # tokenizer
nltk.download('wordnet') # lemmatizer
nltk.download('stopwords') # used to handle words like a, an, the
nltk.download('averaged_perceptron_tagger') # Part of Speech

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
from sklearn.cluster import AgglomerativeClustering



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
url = 'https://github.com/AnoVando/MSIS/raw/master/IA3.csv'
data = pd.read_csv(url, header='infer')
reviews = data['review'].tolist()

In [3]:
print(data[:5])

   id                                             review       label
0   1  About the shop: There is a restaurant in Soi L...  restaurant
1   2  About the shop: Through this store for about t...  restaurant
2   3  Roast Coffee &amp; Eatery is a restaurant loca...  restaurant
3   4  Eat from the children. The shop is opposite. P...  restaurant
4   5  The Ak 1 shop at another branch tastes the sam...  restaurant


In [0]:
custom_list = ['quot', 'ha', 'wa']

Part 1. Topic Model
There are 1000 reviews for restaurants and films in a collection under the attached csv file. All of those
reviews are saved as text files. In this assignment, you are required to investigate the topics of those
reviews. In particular, please follow the steps listed below:
1. Transform those reviews into a term‐document matrix, lemmatize all the words, remove the
stop‐words and punctuations, set the minimal document frequency for each term to be 5 and
include 2‐gram.
2. Use the LDA model to extract the topics of each document. In particular, we assume there are 6
topics.
3. Report the topic distribution and the top‐2 topics of the first 10 restaurant reviews (id = [1:10])
and the first 10 movie reviews (id = [501:510]).
4. Find the top‐5 terms (terms with the top‐5 highest weights) for each of the 6 topics. Based on
those terms, describe what those topics are about.
5. Based on finding in 3 and 4, describe what review 1 [ID=1] and review 501 [ID=501] are about?
Please submit 1 file:
A word file includes python code with your comment #, and one screenshot on your Jupyter
Notebook showing that your code has run through successfully for each of the first four steps (4
screenshots in total). Also, report your answers to question 3, 4, and 5 at the end of the word
file.

1. Transform those reviews into a term‐document matrix, lemmatize all the words, remove the
stop‐words and punctuations, set the minimal document frequency for each term to be 5 and
include 2‐gram.

In [0]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stopwords.words('english'):
            if token not in custom_list:
                result.append(lemmatize_stemming(token))
    return result

In [0]:
processed_docs = data['review'].fillna('').astype(str).map(preprocess)

In [8]:
print(processed_docs)

0      [shop, restaur, soi, langsuan, road, insid, lu...
1      [shop, store, three, year, first, time, tri, r...
2      [roast, coffe, amp, eateri, restaur, locat, se...
3      [eat, children, shop, opposit, phra, prathat, ...
4      [ak, shop, anoth, branch, tast, concentr, tell...
                             ...                        
995    [peopl, aliv, never, die, difficult, know, go,...
996    [first, time, know, chen, tianx, movi, nuclear...
997    [film, time, tear, liter, tear, ya, also, leav...
998    [rememb, child, teacher, alway, take, troubl, ...
999    [abil, episod, mouth, sasuk, year, old, gradua...
Name: review, Length: 1000, dtype: object


In [0]:
dictionary = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [0]:
# Generate TF-IDF Vectors
#processed_tfidf = [" ".join(x) for x in processed]

#tfidf = TfidfVectorizer(ngram_range=(2, 2), min_df=5) # 2-grams and min. document frequency of 5
#tfidf.fit(bow_corpus)
#tfidf = tfidf.transform(bow_corpus)


2. Use the LDA model to extract the topics of each document. In particular, we assume there are 6
topics.

In [27]:
lda_model = gensim.models.LdaModel(bow_corpus, num_topics=6, id2word=dictionary)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.013*"film" + 0.007*"also" + 0.006*"peopl" + 0.005*"make" + 0.005*"love" + 0.005*"stori" + 0.005*"time" + 0.005*"eat" + 0.004*"gt" + 0.004*"like"
Topic: 1 
Words: 0.012*"film" + 0.010*"like" + 0.008*"also" + 0.007*"love" + 0.007*"peopl" + 0.007*"good" + 0.006*"make" + 0.005*"time" + 0.004*"say" + 0.004*"think"
Topic: 2 
Words: 0.012*"film" + 0.007*"love" + 0.006*"peopl" + 0.006*"also" + 0.006*"time" + 0.006*"like" + 0.005*"say" + 0.005*"movi" + 0.004*"one" + 0.004*"see"
Topic: 3 
Words: 0.009*"peopl" + 0.008*"film" + 0.007*"good" + 0.006*"also" + 0.006*"like" + 0.006*"eat" + 0.006*"time" + 0.005*"movi" + 0.005*"love" + 0.005*"one"
Topic: 4 
Words: 0.011*"good" + 0.009*"like" + 0.007*"peopl" + 0.007*"time" + 0.006*"food" + 0.006*"love" + 0.006*"eat" + 0.005*"say" + 0.005*"tast" + 0.005*"make"
Topic: 5 
Words: 0.012*"film" + 0.010*"peopl" + 0.007*"also" + 0.006*"good" + 0.006*"like" + 0.006*"love" + 0.005*"say" + 0.005*"life" + 0.005*"one" + 0.004*"time"


3. Report the topic distribution and the top‐2 topics of the first 10 restaurant reviews (id = [1:10])
and the first 10 movie reviews (id = [501:510]).

In [55]:
for n in range(10):
  print(lda_model[bow_corpus][n])
      

[(4, 0.9940009)]
[(3, 0.9445475), (4, 0.05032585)]
[(3, 0.19050916), (4, 0.8058597)]
[(3, 0.7169321), (4, 0.27477002)]
[(3, 0.87687755), (4, 0.10143518)]
[(3, 0.47085765), (4, 0.5254309)]
[(3, 0.5388849), (4, 0.45329475)]
[(4, 0.9892479)]
[(0, 0.023919497), (1, 0.023957796), (2, 0.023916451), (3, 0.024019271), (4, 0.8802408), (5, 0.02394621)]
[(3, 0.95066565)]


In [59]:
for n in range(510):
  if n >= 500:
      print(lda_model[bow_corpus][n])
  if n > 510:
      break
  n = n + 1

[(1, 0.69372094), (2, 0.3011373)]
[(2, 0.9824048), (4, 0.01547226)]
[(1, 0.0387394), (2, 0.6812616), (5, 0.27911204)]
[(3, 0.9649443)]
[(1, 0.4526614), (2, 0.54581934)]
[(2, 0.9890299)]
[(2, 0.71296626), (5, 0.28189638)]
[(2, 0.9752716)]
[(0, 0.17565808), (2, 0.7278081), (5, 0.09505909)]
[(0, 0.09713469), (1, 0.32730865), (2, 0.2963496), (3, 0.20852555), (5, 0.07020713)]


In [9]:
url = 'https://github.com/AnoVando/MSIS/raw/master/IA3.csv'
data = pd.read_csv(url, header='infer')
data1 = data[0:9]
data2 = data[500:509]
data3 = data1.append(data2)
reviews2 = data3['review'].tolist()

# Tokenize, Lemmatize and Remove Stop Words
lemmatizer = nltk.stem.WordNetLemmatizer()
processed2 = []
for review in reviews2:
    tokens = nltk.word_tokenize(review.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if not token in stopwords.words('english')]
    tokens = [token for token in tokens if not token in STOPWORDS]
    tokens = [token for token in tokens if not token in custom_list]
    processed2.append(tokens)

text = gensim.corpora.Dictionary(processed2)
corpus = [text.doc2bow(doc) for doc in processed2]

lda_model2 = gensim.models.LdaModel(corpus, num_topics=2, id2word=text)

for idx, topic in lda_model2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"film" + 0.007*"like" + 0.007*"good" + 0.006*"taste" + 0.005*"eat" + 0.005*"people" + 0.005*"lone" + 0.005*"baht" + 0.005*"life" + 0.005*"delicious"
Topic: 1 
Words: 0.009*"film" + 0.009*"like" + 0.009*"dolphin" + 0.008*"good" + 0.007*"people" + 0.006*"eat" + 0.006*"time" + 0.006*"price" + 0.005*"restaurant" + 0.005*"life"


In [0]:
# install pyLDAvis if necessary
# !pip install pyLDAvis

In [0]:
# Visualize the topics
#import pyLDAvis
#import pyLDAvis.gensim
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model2, corpus, text)
#vis

4. Find the top‐5 terms (terms with the top‐5 highest weights) for each of the 6 topics. Based on
those terms, describe what those topics are about.

In [12]:
lda_model.show_topics(num_topics=6, num_words=5)

[(0,
  '0.014*"film" + 0.010*"like" + 0.009*"good" + 0.008*"people" + 0.006*"movie"'),
 (1,
  '0.011*"film" + 0.007*"like" + 0.007*"good" + 0.006*"time" + 0.005*"love"'),
 (2,
  '0.010*"film" + 0.009*"good" + 0.008*"people" + 0.006*"love" + 0.006*"time"'),
 (3,
  '0.012*"film" + 0.009*"people" + 0.008*"good" + 0.007*"love" + 0.006*"like"'),
 (4,
  '0.011*"people" + 0.008*"film" + 0.007*"love" + 0.007*"time" + 0.006*"like"'),
 (5,
  '0.011*"film" + 0.009*"like" + 0.008*"people" + 0.008*"time" + 0.006*"movie"')]

Topic 0 is about films and good food.
Topic 1 is about people and films about times they love.
Topic 2 is about people and films about eating good food.
Topic 3 is about people and films about people they like.
Topic 4 is about films that people love.
Topic 5 is about films about loving people.

5. Based on finding in 3 and 4, describe what review 1 [ID=1] and review 501 [ID=501] are about?
Please submit 1 file:
A word file includes python code with your comment #, and one screenshot on your Jupyter
Notebook showing that your code has run through successfully for each of the first four steps (4
screenshots in total). Also, report your answers to question 3, 4, and 5 at the end of the word
file.

In [13]:
print(lda_model2[corpus[1]])
data['review'][0]

[(0, 0.45443976), (1, 0.54556024)]


'About the shop: There is a restaurant in Soi Langsuan (Road) inside of Luxx Hotel. The décor of the restaurant: decorated in a rustic style, white walls, glass tables, red chairs, parquet floors, dim lights open at night, the romantic atmosphere: Duck l&#39;orange Pork Wellington and French onion soup. Average Price: 250-450 Baht Food Review: Duck l&#39;orange (455) Duck breast sliced Pork Wellington (445) is a piece of tender pork stuffed with stuffing and wrapped in a thin pastry and then baked to serve with the sauce. Duck roll (285) is a Duck wrapped with vegetables and dough wrap not much delicious French onion soup (235) Sweet taste Garnish with cheese Bake Scallop (225) is a scallop in a thick cream with cheese. Serve with a thin toast to eat together. Score by topic: &lt;Atmosphere&gt; 8/10 Atmosphere nice romantic &lt;food taste&gt; 7/10&lt;Service&gt; 9/10 Good service, good food recommendation &lt;Value&gt; 6/10 price is quite expensive and about 1,000 baht if not ordered t

In [0]:
# Tokenize, Lemmatize and Remove Stop Words
lemmatizer = nltk.stem.WordNetLemmatizer()
processed = []
for review in reviews:
    tokens = nltk.word_tokenize(review.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if not token in stopwords.words('english')]
    tokens = [token for token in tokens if not token in STOPWORDS]
    tokens = [token for token in tokens if not token in custom_list]
    processed.append(tokens)